# **Programming Assignment 2 - Object Detection + Semantic Segmentation**

#### **Professor**: Dário Oliveira  
#### **Monitora**: Lívia Meinhardt


O objetivo deste trabalho é construir um pipeline de visão computacional que primeiro **detecta** objetos em uma imagem e, em seguida, realiza a **segmentação semântica** em cada objeto detectado. Ou seja, uma segmentação de instância em duas etapas. 

Vocês irão construir e conectar dois modelos distintos:
1.  Um **detector de objetos** (YOLO) para encontrar a localização dos objetos.
2.  Um **segmentador semântico** (U-Net ou outro) para classificar os pixels dentro de cada objeto localizado.

### **Instruções:**

1. **Criação de um Dataset**:  
Vocês usarão o dataset **[PASCAL VOC 2012](https://www.kaggle.com/datasets/gopalbhattrai/pascal-voc-2012-dataset)**. Este conjunto de dados é ideal porque fornece anotações tanto para **caixas delimitadoras** (*bounding boxes*), para a tarefa de detecção, quanto para **máscaras de segmentação** a nível de pixel. 


2. **Implementação e Treinamento da YOLO:**
Sua primeira tarefa é fazer a implementação da YOLOv3, vista em aula. Treine um modelo que recebe uma imagem como entrada e retorna uma lista de predições, onde cada predição contém `(caixa_delimitadora, classe, score_de_confiança)`. Meça o desempenho do seu detector usando a métrica **Mean Average Precision (mAP)**.


3. **Treinar o Segmentador Semântico:**
Sua segunda tarefa é treinar um modelo **U-Net** ou outro de sua preferência para realizar a segmentação. O ponto crucial é que vocês não irão treinar a U-Net com imagens inteiras. Em vez disso, vocês a treinarão com **recortes de imagem (*patches*)** gerados a partir das caixas delimitadoras de referência (*ground-truth*) do dataset. Meça o desempenho do seu segmentador usando a métrica **Average Precision (AP).**

4. **Construir o Pipeline de Inferência:**
    Agora, conecte seus dois modelos treinados. Esta parte envolve escrever um script que executa a tarefa completa de ponta a ponta.

    1.  **Detectar**: Use uma nova imagem de teste e passe-a pelo seu modelo **YOLOv3** treinado para obter uma lista de caixas delimitadoras preditas.
    2.  **Recortar**: Para cada predição de alta confiança do YOLO, **recorte o *patch* da imagem** definido pela caixa delimitadora.
    3.  **Segmentar**: Passe cada *patch* recortado pelo seu modelo **U-Net** treinado para obter uma máscara de segmentação para aquele objeto específico.
    4.  **Combinar**: Crie uma imagem em branco (preta) do mesmo tamanho da imagem original. "Costure" cada máscara gerada de volta nesta imagem em branco, na sua localização original da caixa delimitadora.
    5.  **Visualizar**: Sobreponha a máscara final combinada na imagem original para criar uma visualização final mostrando todos os objetos detectados e segmentados.


5. **Compare com um método *end-to-end:***
Por fim, faça *fine-tuning* do [**Mask R-CNN**](https://docs.pytorch.org/vision/main/models/mask_rcnn.html), um um modelo de segmentação de instância de ponta a ponta (*end-to-end*). Compare o desempenho com o seu pipeline de dois estágios e discuta as diferenças. 


### **Entrega:**

Você deve enviar:

1.  Um **Jupyter Notebook** contendo todo o seu código.
2.  Os **pesos treinados** tanto do seu detector YOLOv3 quanto do seu segmentador U-Net.
3.  Um **relatório ou apresentação** que discuta os desafios e resultados dos seus experimentos. 

## Preparando o dataset pra detecção

In [ ]:
import os
import random
from PIL import Image
import shutil
from ultralytics import YOLO

# Define dataset root
original_dataset_path = 'data'
yolo_dataset_path = 'yolo_dataset'

In [6]:
yolo_dirs = [
    os.path.join(yolo_dataset_path, 'images', 'train'),
    os.path.join(yolo_dataset_path, 'images', 'val'),
    os.path.join(yolo_dataset_path, 'labels', 'train'),
    os.path.join(yolo_dataset_path, 'labels', 'val')
]

for yolo_dir in yolo_dirs:
    os.makedirs(yolo_dir, exist_ok=True)

jpeg_images_dir = os.path.join(original_dataset_path, 'VOC2012_train_val', 'JPEGImages')
annotations_dir = os.path.join(original_dataset_path, 'VOC2012_train_val', 'Annotations')
if not os.path.exists(jpeg_images_dir) or not os.path.exists(annotations_dir):
    raise FileNotFoundError(f"The directory {jpeg_images_dir} or {annotations_dir} does not exist. Please verify the dataset path.")
image_filenames = os.listdir(jpeg_images_dir)
image_ids = [os.path.splitext(filename)[0] for filename in image_filenames if filename.endswith('.jpg')]

random.seed(42)
random.shuffle(image_ids)
split_index = int(0.8 * len(image_ids)) #Spliting the dataset 80% for training, 20% for validation
train_ids = image_ids[:split_index] #taking the first 80% pictures
val_ids = image_ids[split_index:]

FileNotFoundError: The directory /home/pedro/Modelos/Faculdade/DL/Assignment 2/data/VOC2012_train_val/JPEGImages or /home/pedro/Modelos/Faculdade/DL/Assignment 2/data/VOC2012_train_val/Annotations does not exist. Please verify the dataset path.

In [ ]:
import xml.etree.ElementTree as ET 
#this fucntion converts PASCAL_VOC annotations to YOLO format
def create_yolo_annotation(xml_file_path, yolo_label_path, label_dict):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    annotations = [] #list that will store the converted YOLO annotations.

    img_width = int(root.find('size/width').text)
    img_height = int(root.find('size/height').text)

    for obj in root.findall('object'):
        label = obj.find('name').text
        if label not in label_dict:
            continue
        label_idx = label_dict[label]
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        # this is YOLOv8 annotation format: label x_center y_center width height (normalized)
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        annotations.append(f"{label_idx} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    #annotations to the label file
    with open(yolo_label_path, 'w') as f:
        f.write("\n".join(annotations))

In [ ]:
label_dict = {
    'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4,
    'bus': 5, 'car': 6, 'cat': 7, 'chair': 8, 'cow': 9,
    'diningtable': 10, 'dog': 11, 'horse': 12, 'motorbike': 13, 'person': 14,
    'pottedplant': 15, 'sheep': 16, 'sofa': 17, 'train': 18, 'tvmonitor': 19
}

for image_set, ids in [('train', train_ids), ('val', val_ids)]:
    for img_id in ids:
        img_src_path = os.path.join(jpeg_images_dir, f'{img_id}.jpg')
        label_dst_path = os.path.join(yolo_dataset_path, 'labels', image_set, f'{img_id}.txt')

        # Create the YOLO annotation file
        xml_file_path = os.path.join(annotations_dir, f'{img_id}.xml')
        if not os.path.exists(xml_file_path):
            print(f"Warning: Annotation {xml_file_path} not found, skipping.")
            continue
        create_yolo_annotation(xml_file_path, label_dst_path, label_dict)

        # Copy the image to the new YOLO dataset structure
        img_dst_path = os.path.join(yolo_dataset_path, 'images', image_set, f'{img_id}.jpg')
        shutil.copy(img_src_path, img_dst_path)

### Testando na YOLOv8

In [ ]:
yaml_content = f"""
train: images/train
val: images/val

nc: {len(label_dict)}
names: {list(label_dict.keys())}
"""

with open(os.path.join(yolo_dataset_path, 'data.yaml'), 'w') as f:
    f.write(yaml_content)

In [ ]:
model = YOLO('yolov8n.pt')  # Use the YOLOv8 nano model

# Train the model
model.train(
    data=os.path.join(yolo_dataset_path, 'data.yaml'),  # Path to dataset config file
    epochs=2,  # Number of epochs
    imgsz=640,  # Image size
    batch=16,  # Batch size
    name='yolov8_test'  # Experiment name
)

In [ ]:
results = model.predict(source=os.path.join(yolo_dataset_path, 'images/val/'), save=True)  # Run inference on validation set

# Display predictions
for result in results:
    result.show()
    break

In [ ]:
max_conf = 0
best_result = None

for result in results:    
        if (result.boxes.conf.mean().item() > max_conf) and (result.boxes.conf.mean().item() < 0.98) and (result.boxes.conf.shape[0] > 2):
            max_conf = result.boxes.conf.mean().item()
            best_result = result

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

img = best_result.orig_img
boxes = best_result.boxes.xyxy.cpu().numpy()  # xyxy format
classe = best_result.boxes.cls
confianca = best_result.boxes.conf

fig, ax = plt.subplots(1)
ax.imshow(img)

for i, box in enumerate(boxes):
    x1, y1, x2, y2 = box[:4]
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    # Add class and confidence
    class_idx = int(classe[i])
    class_name = best_result.names[class_idx] if hasattr(best_result, 'names') else str(class_idx)
    conf = float(confianca[i])
    ax.text(x1, y1 - 5, f'{class_name}: {conf:.2f}', color='red', fontsize=10, backgroundcolor='white')

plt.show()